# Text Normalization initial prediction Notebook

In [1]:
# initial script
import pandas as pd
import numpy as np
import gc
import re
#Read train dataset CSV
df = pd.read_csv(r'en_train.csv')
df.count()

sentence_id    9918441
token_id       9918441
class          9918441
before         9918390
after          9918395
dtype: int64

#### Create a dictionary of words which store the words that have changed.

In [2]:
from itertools import islice
dfDifference = df[df['before'] != df['after']]
dictionary = dict(zip(dfDifference['before'],dfDifference['after']))
print(list(islice(dictionary.keys(),5)))
print(list(islice(dictionary.values(),5)))
dictionary['1'] = 'one'
dictionary['2'] = 'two'
dictionary['3'] = 'three'
dictionary['4'] = 'four'
dictionary['5'] = 'five'
dictionary['6'] = 'six'
dictionary['7'] = 'seven'
dictionary['8'] = 'eight'
dictionary['9'] = 'nine'

['2006', 'IUCN', '2007', '2008', '91']
['two thousand six', 'i u c n', 'two thousand seven', 'two thousand eight', 'ninety one']


#### Reading the test dataset and setting the column 'y' = 1 for the words present in the dictionary

In [3]:
from itertools import islice
df_test = pd.read_csv(r'en_test.csv')
#df_test = df_test[:1000]
df_test['y'] = 0
df_test['b'] = 0
for k,v in df_test.iterrows():
    if dictionary.get(v['before']) is not None:
        df_test.set_value(k,'y',1)
eg = df_test[df_test['y'] == 1]
print(eg.count())

sentence_id    111810
token_id       111810
before         111802
y              111810
b              111810
dtype: int64


#### Assigning a unique number for the words in the test dataset

In [4]:
unique_col = df_test['before'].unique()

before_list = dict(zip(unique_col, range(1,np.count_nonzero(unique_col)))) 

for k,v in df_test.iterrows():
    if before_list.get(v['before']) is not None:
        num = before_list[v['before']]
        df_test.set_value(k,'b',num)

In [5]:
import statsmodels.api as sm

df_test['intercept'] = 1.0
word_prediction = sm.Logit(df_test['y'], df_test[['b','intercept']]).fit()
word_prediction.summary()

D:\Softwares\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Optimization terminated successfully.
         Current function value: 0.330871
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:              1088564
Model:                          Logit   Df Residuals:                  1088562
Method:                           MLE   Df Model:                            1
Date:                Sat, 16 Dec 2017   Pseudo R-squ.:               0.0004020
Time:                        00:08:02   Log-Likelihood:            -3.6017e+05
converged:                       True   LL-Null:                   -3.6032e+05
                                        LLR p-value:                 5.765e-65
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
b           2.239e-06   1.29e-07     17.320      0.000    1.99e-06    2.49e-06
intercept     -2.1937      0.004   -621.068      0.000      -2.201      -2.187
==============================================================================
"""

#### Predicting the accuracy

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold, cross_val_score, train_test_split
from sklearn import metrics
lr = LogisticRegression(C=1e9)

X = df_test['b']
X = X.reshape(-1, 1)
y = df_test['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
logreg = LogisticRegression()
LRm = logreg.fit(X_train, y_train)
a=metrics.accuracy_score(y_train, LRm.predict(X_test))
print(a)

D:\Softwares\Anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


0.897442869689


In [27]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train,y_train)
#print(y_train.count())
y_pred_class = mnb.predict(df_test['b'][:544282].reshape(-1,1))
print(metrics.accuracy_score(y_test,y_pred_class))

0.897130531599


D:\Softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """


In [7]:
import nltk
train = [(dict(a=1,b=1,c=1), 'y'),(dict(a=1,b=1,c=1), 'x'),(dict(a=1,b=1,c=0), 'y'),(dict(a=0,b=1,c=1), 'x'),(dict(a=0,b=1,c=1), 'y'),(dict(a=0,b=0,c=1), 'y'),(dict(a=0,b=1,c=0), 'x'),
(dict(a=0,b=0,c=0), 'x'),
(dict(a=0,b=1,c=1), 'y')]
train_set = [(1,'y'),(2,'x')]
print(dict(a=1,b=1,c=1))
print(X_train[5:])
#print(dict(X_train[5:].tolist()))
#classifier = nltk.NaiveBayesClassifier.train(train_set)

{'a': 1, 'b': 1, 'c': 1}
[[  202]
 [ 3266]
 [11033]
 ..., 
 [   26]
 [   51]
 [   15]]
